In [1]:
import numpy as np
import netCDF4 as ncdf
import matplotlib as mlab
import matplotlib.pyplot as plt
import xarray as xr
import glob as glob
import os as os
import sys as sys
import pygrib
from metpy.interpolate import log_interpolate_1d
import ray
import psutil
import scipy.signal
import time


#---------------------------------------------------------------------------------------------

class TimerError(Exception):
    """A custom exception used to report errors in use of Timer class"""

class Timer:
    def __init__(self):
        self._start_time = None

    def start(self):
        """Start a new timer"""
        if self._start_time is not None:
            raise TimerError(f"Timer is running. Use .stop() to stop it")

        self._start_time = time.perf_counter()

    def stop(self):
        """Stop the timer, and report the elapsed time"""
        if self._start_time is None:
            raise TimerError(f"Timer is not running. Use .start() to start it")

        elapsed_time = time.perf_counter() - self._start_time
        self._start_time = None
        print(f"Elapsed time: {elapsed_time:0.4f} seconds")

In [19]:
#--------------------------------------------------------------------------------------------------
num_cpus = 1

ray.shutdown()
ray.init(num_cpus=num_cpus)

nz = 50
nx = 1000
ny = 100
data = np.random.rand(nx*ny*nz).reshape(nz,ny,nx)
new  = np.random.rand(nx*ny*nz).reshape(nz,ny,nx)
z    = np.arange(nz)
zi   = np.sort(np.random.rand(nz))*nz

def s(d, z, zi):
    # Do some image processing.
    return np.interp(zi, z, d)

@ray.remote
def f(d, z, zi, j):
    # Do some image processing.
    return np.interp(zi, z, d[:,j])

# Time the serial code below.

toc = time.perf_counter()

for i in np.arange(nx):        
    for j in np.arange(ny):
        new[:,j,i] = s(data[:,j,i], z, zi)
                
tic = time.perf_counter()
print(f"Serial-loop {tic - toc:0.4f} seconds")
  
# Time the Ray code

toc = time.perf_counter()

z_id  = ray.put(z)
zi_id = ray.put(zi)


for i in np.arange(nx): 
    d_id  = ray.put(data[:,:,i])
    raw = ray.get([f.remote(d_id, z_id, zi_id, j) for j in np.arange(ny)])
    # res = np.asarray(raw).transpose()
    # new[:,:,i] = res
                
tic = time.perf_counter()
            
print(f"Ray-loop {tic - toc:0.4f} seconds")



Serial-loop 0.4493 seconds
(f pid=95349) 


(f pid=95348) 


(f pid=95348) 
(f pid=95348) 
(f pid=95349) 


(f pid=95349) 


(f pid=95348) 


(f pid=95347) 
(f pid=95349) 


(f pid=95346) 
(f pid=95347) 
(f pid=95346) 
(f pid=95348) 


(f pid=95347) 
(f pid=95346) 


(f pid=95346) 
Ray-loop 30.6726 seconds


In [41]:

new[10,:,10]

array([0.55820217, 0.69266626, 0.68952211, 0.51199267, 0.51256167,
       0.91806221, 0.78063349, 0.58877353, 0.38301466, 0.16895239,
       0.61272793, 0.71366024, 0.48761593, 0.56135423, 0.4038238 ,
       0.06230081, 0.49236065, 0.50426121, 0.64005067, 0.56576987,
       0.75296683, 0.41345032, 0.34972265, 0.52930709, 0.52072485,
       0.95048878, 0.60386829, 0.10157443, 0.59404696, 0.36443223,
       0.48905468, 0.30951704, 0.56312416, 0.80794414, 0.14013586,
       0.18728921, 0.18315944, 0.67836838, 0.69665304, 0.34149845,
       0.711372  , 0.12777241, 0.16094428, 0.46692   , 0.28028493,
       0.51401584, 0.86582872, 0.79156753, 0.14975324, 0.12928616,
       0.40850552, 0.76917623, 0.28915335, 0.56210656, 0.28178691,
       0.78487054, 0.72211793, 0.13087177, 0.66054264, 0.81285266,
       0.23576482, 0.57608957, 0.30902644, 0.13779929, 0.58492332,
       0.7353831 , 0.83096753, 0.81651363, 0.75270586, 0.7953361 ,
       0.32768386, 0.28237371, 0.60309091, 0.47009208, 0.13917

In [42]:
data[10,:,10]

array([0.45153507, 0.8358469 , 0.56067289, 0.61745126, 0.51480561,
       0.90646475, 0.69481932, 0.7940335 , 0.38424821, 0.15123032,
       0.65005342, 0.76120404, 0.57043396, 0.78355565, 0.46677481,
       0.04505801, 0.28990041, 0.53747808, 0.6508028 , 0.37909974,
       0.7059717 , 0.59070896, 0.28643202, 0.63316579, 0.4684094 ,
       0.96771045, 0.47616278, 0.14533986, 0.6606395 , 0.3109494 ,
       0.36327843, 0.14358963, 0.6462658 , 0.82865683, 0.0856308 ,
       0.11720508, 0.03574809, 0.7524901 , 0.9474496 , 0.38253037,
       0.9029612 , 0.07499792, 0.1750019 , 0.37657453, 0.12116786,
       0.3426218 , 0.97934168, 0.95370929, 0.18236542, 0.00766323,
       0.31151315, 0.745439  , 0.00824558, 0.68272241, 0.20576848,
       0.82512428, 0.96891624, 0.05881733, 0.91527344, 0.88511802,
       0.07599038, 0.52098958, 0.44170415, 0.16321347, 0.58512778,
       0.72709649, 0.75830453, 0.81920196, 0.64802868, 0.78186141,
       0.37526869, 0.12048317, 0.70719946, 0.43171127, 0.05493

In [29]:
print(a[0:100])

[array([0.2971088 , 0.53416018, 0.68853923, 0.82924502, 0.31973357,
       0.38831754, 0.68350646, 0.67033633, 0.19210313, 0.19115954,
       0.44000163, 0.49153693, 0.31100662, 0.16824039, 0.2130362 ,
       0.23989609, 0.22666851, 0.35774232, 0.53972923, 0.50329072,
       0.38686341, 0.25675863, 0.18739579, 0.3162278 , 0.44490285,
       0.67527521, 0.25744666, 0.2724619 , 0.36970647, 0.74571088,
       0.95511843, 0.64750296, 0.16364141, 0.45954534, 0.46047395,
       0.46054791, 0.41400796, 0.51793889, 0.8721831 , 0.99002809,
       0.80935742, 0.77622405, 0.79991557, 0.83265924, 0.69105767,
       0.74577227, 0.41044393, 0.90888085, 0.36823955, 0.22523192]), array([0.3899739 , 0.45283403, 0.30490396, 0.62382189, 0.47829501,
       0.50185235, 0.86907957, 0.86777553, 0.77482591, 0.5462655 ,
       0.47234739, 0.32083248, 0.3936899 , 0.49186523, 0.53635669,
       0.55477718, 0.47747175, 0.81763564, 0.58723433, 0.03227285,
       0.32894266, 0.66055454, 0.83734697, 0.7408399 , 0.60

In [22]:
zi

array([ 1.58133514,  1.58754866,  4.71104162,  5.0822742 ,  5.75282935,
        6.38690732,  8.88657331,  9.05330603,  9.40916043, 12.84473947,
       15.76316392, 16.3209652 , 16.65756689, 16.69607941, 17.24306941,
       18.24429868, 18.36192893, 18.99456117, 19.13649308, 19.13835878,
       21.97914497, 23.47575767, 23.66412435, 24.23253092, 28.35113333,
       29.4599825 , 29.9776224 , 30.17559781, 31.30614993, 31.58359517,
       32.98304919, 33.27688235, 33.46049978, 34.32967684, 36.26827435,
       38.32291209, 38.5506929 , 38.58903802, 39.0869122 , 39.93293314,
       40.24150851, 40.70722606, 41.94896907, 42.47099433, 44.5506257 ,
       45.11976055, 46.33742221, 48.04070792, 48.22918325, 49.66433285])